In [1]:
import os

In [2]:
os.chdir("../")
%pwd

'c:\\Users\\ramak\\OneDrive\\Desktop\\P2\\CatDogClassification'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingModelConfig:
    root_dir: Path
    updated_base_model_path: Path
    trained_model_path: Path
    training_data: Path
    validation_data: Path
    epochs: int
    batch_size: int
    weights: str
    include_top: bool
    input_shape: list
    class_weights: dict

In [4]:
from src.CatDogClassification.constants import *
from src.CatDogClassification.utils.common import read_yaml, create_directory, save_json

In [5]:
class ConfigurationManager:
    def __init__(self, config_filepath = Config_File_Path, param_filepath = Params_File_Path):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(param_filepath)

        create_directory([self.config.main_dir])

    def get_training_model(self) -> TrainingModelConfig:
        training = self.config.training_model
        prepare_base_model = self.config.prepare_base_model
        training_data = os.path.join(self.config.data_ingestion.split_data_dir,"train")
        validation_data = os.path.join(self.config.data_ingestion.split_data_dir,"validation")
        create_directory([training.root_dir])

        prepare_training_model = TrainingModelConfig(
            root_dir = training.root_dir,
            updated_base_model_path = prepare_base_model.updated_base_model_path,
            trained_model_path = training.trained_model_path,
            training_data = Path(training_data),
            validation_data = Path(validation_data),
            epochs = self.params.EPOCHS,
            batch_size = self.params.BATCH_SIZE,
            weights = self.params.WEIGHTS,
            include_top = self.params.INCLUDE_TOP,
            input_shape = self.params.INPUT_SHAPE,
            class_weights = self.params.CLASS_WEIGHTS
            )

        return prepare_training_model

In [6]:
import tensorflow as tf
import numpy as np
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator


[2023-12-15 15:01:07,472: INFO: utils: NumExpr defaulting to 8 threads.]


In [9]:
class Training:
    def __init__(self, config: TrainingModelConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
            # os.path.join("artifacts","prepare_base_model","base_model_updated")
        )



    def train_valid_generator(self):
        # train_data = ImageDataGenerator(rescale = 1./255)
        train_data = ImageDataGenerator(
                                        rescale=1./255,
                                        rotation_range=20,
                                        width_shift_range=0.2,
                                        height_shift_range=0.2,
                                        shear_range=0.2,
                                        zoom_range=0.2,
                                        horizontal_flip=True,
                                        fill_mode='nearest'
                                    )
        test_data = ImageDataGenerator(rescale = 1./255)
        train_generator = train_data.flow_from_directory(self.config.training_data,batch_size=20,target_size=(150,150),class_mode='binary')
        validation_generator = train_data.flow_from_directory(self.config.validation_data,batch_size=20,target_size=(150,150),class_mode='binary')

        self.model.fit(train_generator,
                        epochs=self.config.epochs,
                        validation_data=validation_generator,
                        validation_steps=50,
                        class_weight = self.config.class_weights)
        
        scores = self.model.evaluate(
            validation_generator, batch_size=self.config.batch_size
        )
        # print("Validation Loss:", scores[0])
        # print("Validation Accuracy:", scores[1])
        score = {"loss": scores[0], "accuracy": scores[1]}
        save_json(path=Path("scores.json"), data=score)
        # logger.info("Model Scores saved in scores.json.......")

        self.save_model(path=self.config.trained_model_path, model=self.model)
        # logger.info("Model training and saving completed...........")
    
    

        

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

In [10]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_model()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    # training.model_training()
except Exception as e:
    raise e

[2023-12-15 15:01:32,311: INFO: common: yaml file: C:\Users\ramak\OneDrive\Desktop\P2\CatDogClassification\config\config.yaml loaded]
[2023-12-15 15:01:32,317: INFO: common: yaml file: C:\Users\ramak\OneDrive\Desktop\P2\CatDogClassification\params.yaml loaded]
[2023-12-15 15:01:32,321: INFO: common: Created directory at: artifacts]
[2023-12-15 15:01:32,323: INFO: common: Created directory at: artifacts/training_model]
Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Epoch 1/20
100/100 [==============================] - 278s 3s/step - loss: 0.8407 - accuracy: 0.5615 - val_loss: 0.5654 - val_accuracy: 0.6860
Epoch 2/20
100/100 [==============================] - 277s 3s/step - loss: 0.6882 - accuracy: 0.7030 - val_loss: 0.5013 - val_accuracy: 0.7490
Epoch 3/20
100/100 [==============================] - 279s 3s/step - loss: 0.6166 - accuracy: 0.7705 - val_loss: 0.4478 - val_accuracy: 0.7970
Epoch 4/20
100/100 [==============================] - 277s 3s/ste

C:\Users\ramak\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
